### Tracer les périmètres des bureaux de vote à partir des adresses des électeurs 
- Traitement de l'extrait anonymisé du Répertoire électoral unique, publié par l'Insee (version 27/06/2023)
https://www.data.gouv.fr/fr/datasets/bureaux-de-vote-et-adresses-de-leurs-electeurs/ 

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.9f' % x)
from pathlib import Path
import numpy as np
import geopandas as gpd
from shapely.ops import unary_union
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords

#### Préparation des données

In [2]:
# Chargement de l'extrait anonymisé du REU : 
adresses=pd.read_csv("Data_REU_Etalab/table-adresses-reu.csv")

adresses["id_brut_bv_reu"]=adresses["id_brut_bv_reu"].astype("str")
adresses["BV"]=adresses["id_brut_bv_reu"].str.split("_").str[-1]
adresses["code_commune_ref"]=adresses["code_commune_ref"].astype("str")
adresses.loc[adresses["code_commune_ref"].str.len()==4, "INSEE_COM"]="0"+adresses["code_commune_ref"]
adresses.loc[adresses["code_commune_ref"].str.len()==5, "INSEE_COM"]=adresses["code_commune_ref"]

adresses.rename(columns={"id_brut_bv_reu":"CODE_BV"}, inplace=True)

/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Nettoyage des adresses

In [3]:
# Sélection des communes constituées d'un seul bureau de vote :
Mono_BV=adresses[adresses["nb_bv_commune"]==1]
Com_MonoBV=Mono_BV["INSEE_COM"].unique().tolist()
# Les contours de ces bureaux de vote uniques seront tout simplement les limites de la commune.

# Sélection des communes qui comprennent plusieurs bureaux de vote :
Multi_BV=adresses[adresses["nb_bv_commune"]!=1]
Com_MultiBV=Multi_BV["INSEE_COM"].unique().tolist()

# Si plusieurs points d'adresses partagent à la fois les mêmes coordonnées géographiques et le même bureau de vote, 
# on les fusionne en additionnant leur nombre d'adresse, 
Doublons=Multi_BV[Multi_BV.duplicated(subset=["longitude","latitude","CODE_BV"], keep=False)]
Doublons=Doublons.groupby(["CODE_BV","longitude","latitude"]).agg({"nb_adresses": "sum","INSEE_COM":"first"}).reset_index(drop=False)
Multi_BV.drop_duplicates(subset=["longitude","latitude","CODE_BV"], keep=False, inplace=True)
Multi_BV=pd.concat([Multi_BV,Doublons])

# Si plusieurs points d'adresses partagent les mêmes coordonnées géographiques mais sont rattachés à des bureaux de vote différents,
# ça crée une ambiguité lors de la création des diagrammes de voronoï. 
# On choisit donc de supprimer les lignes correspondantes qui rassemblent le moins d'adresse (variable "nb_adresse") 
# Ce qui supprime 521 308 lignes d'adresses, soit 4,6% du fichier des communes à plusieurs bureaux de vote
Conflits=Multi_BV[Multi_BV.duplicated(subset=["longitude","latitude"], keep=False)]
Conflits["LatLong"]=Conflits["latitude"].astype("str")+"_"+Conflits["longitude"].astype("str")
ConflitsGroup=Conflits.groupby(["LatLong"]).agg({"nb_adresses": "sum"}).reset_index(drop=False)
ConflitsGroup.rename(columns={"nb_adresses":"nb_total_adresses"}, inplace=True)

Multi_BV["LatLong"]=Multi_BV["latitude"].astype("str")+"_"+Multi_BV["longitude"].astype("str")
Multi_BV=Multi_BV.merge(ConflitsGroup, on="LatLong", how="left")

Multi_BV.loc[Multi_BV["nb_total_adresses"].isna(), "nb_total_adresses"]=Multi_BV["nb_adresses"]
Multi_BV=Multi_BV[Multi_BV["nb_adresses"]>(Multi_BV["nb_total_adresses"]/2)]

Multi_BV=Multi_BV[['CODE_BV','INSEE_COM','nb_adresses','nb_bv_commune','longitude','latitude']]

# Cette opération peut créer des erreurs à la marge, en supprimant des points d'adresses de bureaux de vote entier, ce qui bloque l'étape voronoï
# On identifie ces communes à problèmes :
BV_Communes=Multi_BV.groupby("INSEE_COM").agg({"nb_bv_commune": "first", "CODE_BV": "nunique"}).reset_index(drop=False)
BV_Communes.rename(columns={"nb_bv_commune":"Nb_BV_theorique", "CODE_BV":"Nb_BV_disponible"}, inplace=True)
BV_Communes["Diff"]=BV_Communes["Nb_BV_theorique"]-BV_Communes["Nb_BV_disponible"]

# Ce qui permet de lister les 9 communes comprenant plusieurs bureaux de vote en théorie, mais avec un seul bureau dans les données,
Erreurs=BV_Communes[(BV_Communes["Nb_BV_disponible"]==1)]
Com_Erreurs=Erreurs["INSEE_COM"].unique().tolist()
# Les autres communes générant des erreurs lors des traitements sont ajoutées ici : 
#Com_Bug=['97502','30045','2A141','13118','13002', '13051', '13052', '13119']
# On exporte les points d'adresses qui correspondent à ces communes, afin de les découper ultérieurement dans Qgis
Multi_BV_Erreurs=Multi_BV[Multi_BV["INSEE_COM"].isin(Com_Erreurs)]
Multi_BV_Erreurs.to_csv("adresses_communes_erreurs_BV.csv")
# Et enfin on retire les points d'adresse de ces communes du dataframe à traiter :
Multi_BV=Multi_BV[~Multi_BV["INSEE_COM"].isin(Com_Erreurs)]

# On identifie également les 16 communes dont le nombre de bureau de vote présents dans les données est inférieur au nombre théorique 
# (il s'agit parfois de bureaux "complémentaires", réunissant des électeurs qui n'ont pas de domicile dans la commune)
Alerte=BV_Communes[(BV_Communes["Diff"]>=1) & (BV_Communes["Nb_BV_disponible"]>=2)]
Com_Alerte=Alerte["INSEE_COM"].unique().tolist()
# ['06151', '13005', '13209', '13211', '13214', '33165', '34123', '51612', '52140', '75110', '79196', '89086', '97120', '97228', '97353', '97415']


/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_55066/919876716.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Conflits["LatLong"]=Conflits["latitude"].astype("str")+"_"+Conflits["longitude"].astype("str")


### Préparation du découpage

In [4]:
# Convertir la liste des adresses géocodées en fichier géographique :
Points_Adresses=gpd.GeoDataFrame(Multi_BV, geometry=gpd.points_from_xy(Multi_BV.longitude, Multi_BV.latitude), crs="EPSG:4326")
Points_Adresses = Points_Adresses[["INSEE_COM","CODE_BV","nb_bv_commune","nb_adresses","geometry"]]

# importer les délimitations des communes :
# Source : IGN - Admin Express édition Juin 2022 https://geoservices.ign.fr/adminexpress 
communes="Communes_Admin_Express_2022/COMMUNE.shp"
com=gpd.read_file(communes)[["INSEE_COM","geometry","NOM"]]
# Ajouter les arrondissements de Paris, Lyon et Marseille :
Paris_Lyon_Marseille=["75056","69123","13055"]
com.drop(com[com["INSEE_COM"].isin(Paris_Lyon_Marseille)].index, inplace=True)
arrond=gpd.read_file("Communes_Admin_Express_2022/ARRONDISSEMENT_MUNICIPAL.shp")[["INSEE_ARM","geometry","NOM"]]
arrond.rename(columns={"INSEE_ARM":"INSEE_COM"}, inplace=True)
com_all=pd.concat([com, arrond])
com_all.rename(columns={"NOM":"COMMUNE"}, inplace=True)

# Adapter le système de projection pour permettre un traitement par Geovoronoi (reprojection en World Mercator EPSG:3395) :
com_all = com_all.to_crs("EPSG:3395")
Points_Adresses = Points_Adresses.to_crs(com_all.crs)

### Boucle de découpage des départements en bureaux de votes

In [9]:
# Liste de tous les départements :
Depart=com_all["INSEE_COM"].str[:2].unique().tolist()
# Départements déjà traités :
#   Depart_done=[]
# Mise à jour de la liste des départements à traiter :
#   Depart = [e for e in Depart if e not in (Depart_done)]

for departement in Depart: 
    # Filtrer les adresses des communes de plusieurs bureaux de votes, inclues dans le département :
    Dep=Points_Adresses[Points_Adresses["INSEE_COM"].str.startswith(departement)]
    # Lister les code Insee de ces communes : 
    communes_multiBV=Dep["INSEE_COM"].unique().tolist()

    # Lister les code Insee des communes de ce département qui ne comprennent qu'un seul bureau de vote : 
    monoBV=adresses[adresses["INSEE_COM"].str.startswith(departement)]
    monoBV=monoBV[monoBV["nb_bv_commune"]==1]
    monoBV["code_commune_ref"]=monoBV["code_commune_ref"].astype("str")
    monoBV.loc[monoBV["code_commune_ref"].str.len()==4, "INSEE_COM"]="0"+monoBV["code_commune_ref"]
    monoBV.loc[monoBV["code_commune_ref"].str.len()==5, "INSEE_COM"]=monoBV["code_commune_ref"]
    communes_monoBV=monoBV["INSEE_COM"].unique().tolist()

    # Dans le périmètre de chaque commune, tracer les diagrammes de voronoi autour de chaque électeur, 
    # puis les fusionner par bureau de vote et les exporter dans un fichier geojson :
    for insee in communes_multiBV:
        # sélection du contour communal et des adresses géolocalisées de la commune traitée :
        com=com_all[com_all["INSEE_COM"]==insee]
        elect=Dep[Dep["INSEE_COM"]==insee]
        # Préparation des données du périmètre de la commune: 
        boundary_shape = unary_union(com.geometry)
        # Un filtre qui exclut les adresses se trouvant en dehors du périmètre de la commune (sinon cause d'erreur qui stoppe la boucle)
        elect=elect[elect.geometry.within(boundary_shape)]
        # Préparation des données des adresses :
        coords = points_to_coords(elect.geometry)
        # création des diagrammes de Voronoï
        poly_shapes,poly_to_pt_assignments = voronoi_regions_from_coords(coords, boundary_shape)
        # on transforme les données obtenues en geodataframe :
        vor = gpd.GeoSeries(poly_shapes, crs="EPSG:3395")
        voronoi = gpd.GeoDataFrame(vor, columns=["geometry"],crs="EPSG:3395")
        # On reprojette les polygones voronoï et les points d'adresses en WGS84 :
        voronoi_ok=voronoi.to_crs("EPSG:4326")
        elect_ok=elect.to_crs("EPSG:4326")
        # On fait une jointure spatiale pour attribuer le code de bureau de vote et le code insee à chaque polygone Voronoï :
        result=voronoi_ok.sjoin(elect_ok)
        # Idéalement, à ce stade, on fusionne les polygones de Voronoi partageant le même CODE_BV
        # mais trop d'erreurs liées à des "géométries invalides". On passera donc par Mapshaper, plus fiable pour cette opération.
        # bvote=result.dissolve(by="CODE_BV")
        result.to_file("Export_Voronoi/"+insee+"_voronoi_bvote.geojson")

    # Rassembler tous les fichiers Voronoi du département :
    from pathlib import Path
    folder = Path("Export_Voronoi/")
    voronoi = folder.glob("*_voronoi_bvote.geojson")
    bvote_all = pd.concat([
        gpd.read_file(shp)
        for shp in voronoi
    ]).pipe(gpd.GeoDataFrame)
    bvote_all=bvote_all.to_crs("EPSG:4326")
    # Ajouter les contours des communes comprenant un seul bureau de vote : 
    com_monoBV = com_all.to_crs("EPSG:4326")
    com_monoBV_Dep=com_monoBV[com_monoBV["INSEE_COM"].isin(communes_monoBV)]
    com_monoBV_Dep["CODE_BV"]=com_monoBV_Dep["INSEE_COM"]+"_1"

    Bvote_Dep=pd.concat([bvote_all, com_monoBV_Dep])
    Bvote_Dep.drop(columns={"index_right"}, inplace=True)
    Bvote_Dep["INSEE_COM"]=Bvote_Dep["INSEE_COM"].str[:5]
    Bvote_Dep["DEP"]=Bvote_Dep["INSEE_COM"].str[:2]
    Noms_com=com_all[["INSEE_COM","COMMUNE"]]
    Bvote_Dep=Bvote_Dep.merge(Noms_com, on="INSEE_COM", how="left")
    Bvote_Dep.drop(columns={"COMMUNE_x"}, inplace=True)
    Bvote_Dep.rename(columns={"COMMUNE_y":"COMMUNE"}, inplace=True)
    Bvote_Dep=Bvote_Dep[["COMMUNE","DEP","INSEE_COM","CODE_BV","nb_bv_commune","geometry"]]
    Bvote_Dep.to_file("Export_MultiBV/Bvote_Voronoi_Dep_"+departement+".geojson")

    # Exécuter les commandes Mapshaper dans le makefile pour fusionner les diagrammes de voronoi
    # supprimer les ilôts de moins de 0,005km2, simplifier les contours à 15%,
    # puis exporter un fichier geojson des bureaux de vote du département :
    ! make BVote NOM=$departement
    ! make simplifier NOM=$departement
    # Vider le dossier Export-Voronoi :
    ! make clean



/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


mapshaper -i Export_MultiBV/Bvote_Voronoi_Dep_04.geojson \
	-dissolve2 fields=CODE_BV copy-fields=COMMUNE,INSEE_COM,DEP,nb_bv_commune \
	-o Export_Propre/Bvote_Propre_Dep_04.geojson
[dissolve2] Removed 265 / 266 slivers using 280+ sqm variable threshold
[dissolve2] Dissolved 21,442 features into 261 features
[o] Wrote Export_Propre/Bvote_Propre_Dep_04.geojson
mapshaper -i Export_Propre/Bvote_Propre_Dep_04.geojson -simplify 15% -filter-slivers min-area="0.005km2" -o force Export_Propre/Bvote_Propre_Dep_04.geojson
[simplify] Repaired 3 intersections
[filter-slivers] Removed 298 slivers using 5000+ sqm variable threshold
[o] Wrote Export_Propre/Bvote_Propre_Dep_04.geojson
rm Export_Voronoi/*.geojson


/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


mapshaper -i Export_MultiBV/Bvote_Voronoi_Dep_29.geojson \
	-dissolve2 fields=CODE_BV copy-fields=COMMUNE,INSEE_COM,DEP,nb_bv_commune \
	-o Export_Propre/Bvote_Propre_Dep_29.geojson
[dissolve2] Removed 3,741 / 3,748 slivers using 180+ sqm variable threshold
[dissolve2] Dissolved 242,248 features into 831 features
[o] Wrote Export_Propre/Bvote_Propre_Dep_29.geojson
mapshaper -i Export_Propre/Bvote_Propre_Dep_29.geojson -simplify 15% -filter-slivers min-area="0.005km2" -o force Export_Propre/Bvote_Propre_Dep_29.geojson
[simplify] Repaired 8 intersections
[filter-slivers] Removed 1,473 slivers using 5000+ sqm variable threshold
[o] Wrote Export_Propre/Bvote_Propre_Dep_29.geojson
rm Export_Voronoi/*.geojson


/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


mapshaper -i Export_MultiBV/Bvote_Voronoi_Dep_06.geojson \
	-dissolve2 fields=CODE_BV copy-fields=COMMUNE,INSEE_COM,DEP,nb_bv_commune \
	-o Export_Propre/Bvote_Propre_Dep_06.geojson
[dissolve2] Removed 2,261 / 2,263 slivers using 170+ sqm variable threshold
[dissolve2] Dissolved 105,070 features into 981 features
[o] Wrote Export_Propre/Bvote_Propre_Dep_06.geojson
mapshaper -i Export_Propre/Bvote_Propre_Dep_06.geojson -simplify 15% -filter-slivers min-area="0.005km2" -o force Export_Propre/Bvote_Propre_Dep_06.geojson
[simplify] Repaired 6 intersections
[filter-slivers] Removed 1,656 slivers using 5000+ sqm variable threshold
[o] Wrote Export_Propre/Bvote_Propre_Dep_06.geojson
rm Export_Voronoi/*.geojson


/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


mapshaper -i Export_MultiBV/Bvote_Voronoi_Dep_08.geojson \
	-dissolve2 fields=CODE_BV copy-fields=COMMUNE,INSEE_COM,DEP,nb_bv_commune \
	-o Export_Propre/Bvote_Propre_Dep_08.geojson
[dissolve2] Removed 702 / 706 slivers using 260+ sqm variable threshold
[dissolve2] Dissolved 36,398 features into 559 features
[o] Wrote Export_Propre/Bvote_Propre_Dep_08.geojson
mapshaper -i Export_Propre/Bvote_Propre_Dep_08.geojson -simplify 15% -filter-slivers min-area="0.005km2" -o force Export_Propre/Bvote_Propre_Dep_08.geojson
[filter-slivers] Removed 174 slivers using 5000+ sqm variable threshold
[o] Wrote Export_Propre/Bvote_Propre_Dep_08.geojson
rm Export_Voronoi/*.geojson


/Users/denisvannier/opt/anaconda3/envs/DATAXPLORE/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


mapshaper -i Export_MultiBV/Bvote_Voronoi_Dep_42.geojson \
	-dissolve2 fields=CODE_BV copy-fields=COMMUNE,INSEE_COM,DEP,nb_bv_commune \
	-o Export_Propre/Bvote_Propre_Dep_42.geojson
